In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
from langchain_groq import ChatGroq

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [2]:
llm = ChatGroq(temperature=0.5, groq_api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")


In [3]:
# def generate_viva_questions(topics):
#     chat_history = []

#     prompt = ChatPromptTemplate.from_messages([
#         ("system", """
#         You are a chatbot that will provide one popular viva or oral exam question on the topic or topics - {topics}.
#         Provide only one single question and nothing else.
#          If multiple topics are provided, you can provide a question on any of the topics.
#          Keep the question simple and straightforward and avoid complex questions.
#          The question difficulty should be easy to medium.

#         """),
#         MessagesPlaceholder(variable_name="chat_history"),
#         ("human","Give me a viva question on {topics}")
#     ])

#     chain = prompt|llm

#     response = chain.invoke({"topics":topics,"chat_history":chat_history})
#     chat_history.append(AIMessage(response.content))
#     chat_history.append(HumanMessage("Now give me another question on the same topic"))
#     return response.content, chat_history

In [4]:
# result = generate_viva_questions("mvc architecture, react manifesto, aglie methodology")

In [5]:
# result[0]

In [6]:
# result[1]   

In [10]:
# Entire code for generating viva questions, answers and feedback

import os
from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq

# Load the GROQ API key from environment variables
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Initialize the ChatGroq instance
llm = ChatGroq(temperature=0.5, groq_api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")

def generate_viva_questions(topics):
    chat_history = []

    prompt_question = ChatPromptTemplate.from_messages([
        ("system", """
        You are a chatbot that will provide one popular viva or oral exam question on the topic or topics - {topics}.
        Provide only one single question and nothing else.
        You never produce same question twice.
        If multiple topics are provided, you can provide a question on any of the topics.
        Keep the question simple and straightforward and avoid complex questions.
        The questions should be theorectical majorly and do not ask to write code.
        The question difficulty should be easy to medium.
        """),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "Give me a viva question on {topics}")
    ])

    chain_q = prompt_question | llm

    # Generate the first question automatically
    response_q = chain_q.invoke({"topics": topics, "chat_history": chat_history})
    response_q_content = response_q.content
    chat_history.append(AIMessage(response_q.content))
    print(f"Question: {response_q.content.strip()}")
    generate_viva_answer(response_q, chat_history)

def generate_viva_answer(response_q_content, chat_history):
    prompt_answer = ChatPromptTemplate.from_messages([
        ("system", """
        Answer the question - {response_q_content} - that was generated.
        Answer the question in a simple and straightforward manner like you would in a viva or oral exam in a paragraph manner.
        The answer should be concise and to the point.
        The answer to the question should not be more than 4-5 sentences.
        Give the output as Answer: followed by the answer only.
        """),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "Give the answer to the question {response_q_content}")
    ])

    chain_a = prompt_answer | llm

    # Generate the first question automatically
    response_a = chain_a.invoke({"response_q_content": response_q_content, "chat_history": chat_history})
    response_a_content = response_a.content
    chat_history.append(AIMessage(response_a.content))
    print(f"{response_a.content.strip()}")
    user_answer = input("Enter your answer: ")
    generate_feedback(response_q_content,response_a_content, user_answer, chat_history)

def generate_feedback(response_q_content, response_a_content,user_answer, chat_history):
    prompt_feedback = ChatPromptTemplate.from_messages([
        ("system", """
        Provide feedback on the answer given by the user ie {user_answer}.
        Compare the user's answer that is {user_answer} with the answer generated by the chatbot- {response_a_content} which is on the same question- {response_q_content}.
        See whether the users answer is correct or not by comparing it with the chatbot's answer - the meaning should be similar.
        Generate two scrores- literal score(whether the same words were used or not) and semantic score(whether the meaning is the same or not).
        Both these scores should be in percentage.
        Present your answer in a paragraph manner.
        So your answer should include the following things:
           1.Whether the student's answer is correct or not. If not correct then give the correct answer
           2.Literal score in percentage
           3.Semantic score in percentage
           4. Any additional comments on what the student could have done better
         
        These 4 points should be presented in a paragraph manner but keep it short and concise.
        Talk as if you are a teacher and be polite while giving feedback to a student in a viva or oral exam. directly address the student as 'you'.
        """),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "Provide feedback on the answer given by the user-{user_answer}")
    ])

    chain_f = prompt_feedback | llm

    # Generate the first question automatically
    response_f = chain_f.invoke({"response_q_content": response_q_content, "response_a_content": response_a_content, "user_answer": user_answer, "chat_history": chat_history})
    response_f_content = response_f.content
    chat_history.append(AIMessage(response_f.content))
    print(f"{response_f.content.strip()}")

    # Ask the user if they want to continue
    user_continue = input("Do you want to continue?. Type 'no' if you want to stop: ")
    if user_continue.lower() == "no":
        print("Thank you for using the chatbot!")
    else:
        generate_viva_questions("Machine Learning, Artificial Intelligence")


# Example usage
topics = "machine learning, artificial intelligence"
generate_viva_questions(topics)


Question: Explain the difference between overfitting and underfitting in machine learning models, and provide a scenario where each occurs.
Answer: 
Overfitting and underfitting are two common issues that can occur in machine learning models. Overfitting occurs when a model is too complex and learns the noise in the training data, resulting in poor performance on new, unseen data. For example, consider a model that is trained to predict house prices based on various features such as number of bedrooms, square footage, and location. If the model includes too many features and becomes overly complex, it may overfit the training data and perform poorly on new data. On the other hand, underfitting occurs when a model is too simple and fails to capture the underlying patterns in the data, resulting in poor performance on both training and test data. A scenario where underfitting occurs is when a linear regression model is used to predict a non-linear relationship between two variables.
You'

In [ ]:
prompt = """
    You are talking to a 5 year old and make crying sounds everynow and then.
    """
    # Chat history is initialized every time the app or this function is invoked
chat_history = [{"role": "system", "content": prompt}]


def get_llm_response(question):
        # Add the user's question to the chat history
        chat_history.append({"role": "user", "content": question})

        # Invoke the language model (replace with your actual LLM invocation logic)
        llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")
        response = llm.invoke(chat_history)

        # Append the assistant's response to the chat history
        chat_history.append({"role": "assistant", "content": response.content})

        return response.content

    return get_llm_response